# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [93]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data/*'

# Create a for loop to create a list of files and collect each filepath
# for root, dirs, files in os.walk(filepath):
# # join the file path and roots with the subdirectories using glob
#     file_path_list = (glob.glob(os.path.join(root,'*')))
#     #print(file_path_list)
# print(file_path_list)

file_path_list = glob.glob(filepath)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [98]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [99]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


Our processed data are saved with following columns:

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [2]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [3]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

## Table Definition and Pipeline Queries

#### Table 1 - Session_song_info
This table keeps the song information of the given session id. The primary keys are sessionId and iteminsession because the combination of the two define a unique song that was played. Partitioning of the song data is done by sessionId and itemInSession.

In [5]:
# Table creation of session_song_info
create1 = """
CREATE TABLE IF NOT EXISTS session_song_info
(artist_name text, song_name text, song_length float, sessionId int, itemInSession int, 
PRIMARY KEY (sessionId, itemInSession))
"""

# Table insersion of session_song_info
insert1 = "INSERT INTO session_song_info \
                    (artist_name, song_name, song_length, sessionId, itemInSession)"
insert1 = insert1 + " VALUES (%s, %s, %s, %s, %s)"

# Select the song details for the given session id and the item
select1 = """
SELECT artist_name, song_name, song_length 
FROM session_song_info
WHERE sessionId = 338 AND itemInSession = 4
""" 

#### Table 2 - song_playlist_session

This table keeps track of the song played by the user in the given session. The primary key is composed of 2 partition key, userId and sessionId which defines one unique song, and the clustering key itemInSesion.

In [6]:
# Create query for the table song_playlist_session
create2 = """
CREATE TABLE IF NOT EXISTS song_playlist_session
(artist_name text, song_name text, firstName text, lastName text, userId int, sessionId int, itemInSession int,
PRIMARY KEY ((userId, sessionId), itemInSession))
"""

# Table insersion for song_playlist_session
insert2 = "INSERT INTO song_playlist_session \
                    (artist_name, song_name, firstName, lastName, userId, sessionId, itemInSession)"
insert2 = insert2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

# Select statement for song_playlist_session
select2 = """
SELECT artist_name, song_name, firstName, lastName 
FROM song_playlist_session
WHERE userId = 10 AND sessionId = 182
"""

#### Table 3 - song_listner_info
This table keeps track of which user listened to which song to view the preference by each users. The primary key is the song_name and userid. The preference data is saved if the user has listened to the song at least once.

In [13]:
# Create the table song_lisnter_info
create3 = """
CREATE TABLE IF NOT EXISTS song_listener_info
(song_name text, firstName text, lastName text, userId int,
PRIMARY KEY (song_name, userId))
"""
# insertion query for the table song_listner_info
insert3 = "INSERT INTO song_listener_info \
        (song_name, firstName, lastName, userId)"
insert3 = insert3 + " VALUES (%s, %s, %s, %s)"
# select query for the song_lisnter_info
select3 = """
SELECT firstName, lastName 
FROM song_listener_info
WHERE song_name = 'All Hands Against His Own'
"""

In [14]:
# Collect the queries, create the tables
create_queries = [create1, create2, create3]
select_queries = [select1, select2, select3]

for query in create_queries:
    try:
        session.execute(query)
    except Exception as e:
        print(e)

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(insert1, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        session.execute(insert2, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))
        session.execute(insert3, (line[9], line[1], line[4], int(line[10])))

#### Select query showcase

In [16]:
# Find the song information 
# with sessionId = 338 AND itemInSession = 4
try:
    rows = session.execute(select1)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song_name, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [19]:
# Find the song listener information 
# with userId = 10 AND sessionId = 182
try:
    rows = session.execute(select2)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song_name, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [20]:
# Find the song listner name for
# with song_name = 'All Hands Against His Own'
try:
    rows = session.execute(select3)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [22]:
drop1 = "DROP TABLE session_song_info"
drop2 = "DROP TABLE song_playlist_session"
drop3 = "DROP TABLE song_listener_info"
drop_queries = [drop1, drop2, drop3]
for query in drop_queries:
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()